In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost

In [ ]:
from collections import defaultdict
import pandas as pnd
from tqdm.notebook import tqdm
import networkx as nx
import numpy as np
import pandas as pd
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error
from scipy.sparse.csgraph import connected_components

In [ ]:
os.system(' cp "drive/My Drive/train_dataset_VK.zip" "train_dataset_VK.zip" ')
os.system(' unzip train_dataset_VK.zip' )

0

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
indd = np.split(test.index.values, np.unique(test['ego_id'].values, return_index=True)[1][1:])

In [ ]:
xxx = test.ego_id.unique()
tk0 = tqdm(enumerate(xxx), total = len(xxx))
list_feats1 = []
list_feats2 = []
list_feats3 = []
list_feats4 = []
list_feats5 = []
list_feats6 = []
list_feats7 = []
list_feats8 = []
list_feats9 = []
list_feats10 = []

list_preds = []
list_true = []
for i, num_ in tk0:
    tmp_df = test.loc[indd[i], :]
    tmp_train_df = tmp_df[~tmp_df.x1.isnull()]

    tmp_train_df = tmp_train_df[tmp_train_df.x1 > tmp_train_df.x1.mean() * 0.8]

    tmp_val_df = tmp_df[tmp_df.x1.isnull()]

    ind1 = tmp_train_df['u'].tolist() + tmp_train_df['v'].tolist()
    ind2 = tmp_train_df['v'].tolist() + tmp_train_df['u'].tolist()
    val  = tmp_train_df['x1'].tolist() + tmp_train_df['x1'].tolist()

    mx = max(tmp_df['u'].max(), tmp_df['v'].max())


    friend_matrix = csr_matrix((val, (ind1, ind2)), 
                            shape=(mx + 1, mx + 1))
    
    sums = friend_matrix.T.sum(axis=1).A1
    counts = np.diff(friend_matrix.indptr)
    averages = np.divide(sums, counts, out=np.zeros_like(sums), where=counts!=0) * 0.5

    sums = friend_matrix.sum(axis=1).A1
    counts = np.diff(friend_matrix.T.indptr)
    averages_add = np.divide(sums, counts, out=np.zeros_like(sums), where=counts!=0) * 0.5

    friend_matrix = csr_matrix(friend_matrix.multiply(friend_matrix > averages))
    friend_matrix.eliminate_zeros()

    friend_matrix_ones = friend_matrix.copy()
    friend_matrix_ones.data = np.ones(len(friend_matrix_ones.data))
    

    mult_ones1 = friend_matrix_ones * friend_matrix_ones.T
    mult1 = (friend_matrix * friend_matrix_ones.T) 
    i1, i2 = mult1.nonzero()
    a = mult1.data
    b = np.array(mult_ones1[i1, i2])[0] 
    mult1.data = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
    val_ind1 = tmp_val_df['u'].values
    val_ind2 = tmp_val_df['v'].values
    # val_val  = tmp_val_df['target'].values
    # preds1 = (mult1[val_ind1, val_ind2] + mult1[val_ind2, val_ind1] ) / 2
    
    mult_ones2 = friend_matrix_ones.T * friend_matrix_ones
    mult2 = (friend_matrix.T * friend_matrix_ones) 
    i1, i2 = mult2.nonzero()
    a = mult2.data
    b = np.array(mult_ones2[i1, i2])[0] 
    mult2.data = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
    val_ind1 = tmp_val_df['u'].values
    val_ind2 = tmp_val_df['v'].values
    # val_val  = tmp_val_df['target'].values
    # preds2 = (mult2[val_ind1, val_ind2] + mult2[val_ind2, val_ind1] ) / 2

    list_feats1 += mult1[val_ind1, val_ind2].tolist()[0]
    list_feats2 += mult2[val_ind1, val_ind2].tolist()[0]

    list_feats3 += mult1[val_ind2, val_ind1].tolist()[0]
    list_feats4 += mult2[val_ind2, val_ind1].tolist()[0]

    list_feats5 += mult_ones1[val_ind1, val_ind2].tolist()[0]
    list_feats6 += tmp_val_df.index.tolist()

    list_feats7 += averages[val_ind1].tolist()
    list_feats8 += averages[val_ind2].tolist()

    list_feats9 += averages_add[val_ind1].tolist()
    list_feats10 += averages_add[val_ind2].tolist()
    # preds = (preds1 + preds2) / 2
    # list_true += val_val.tolist()
    # list_preds += (np.array(preds)[0] ).tolist()

  0%|          | 0/20596 [00:00<?, ?it/s]

In [ ]:
import pickle
with open('drive/MyDrive/vkfeats1test.pickle', 'wb') as f:
    pickle.dump([list_feats1, list_feats2, list_feats3, list_feats4, list_feats5, list_feats6, list_feats7, list_feats8, list_feats9, list_feats10], f)

In [ ]:
# len(list_feats1)

In [ ]:
xxx = test.ego_id.unique()
tk0 = tqdm(enumerate(xxx), total = len(xxx))
list_feats1 = []
list_feats2 = []
list_feats3 = []
list_feats4 = []
list_feats5 = []
list_feats6 = []
list_feats7 = []
list_feats8 = []
list_feats9 = []
list_feats10 = []

list_preds = []
list_true = []
for i, num_ in tk0:
    tmp_df = test.loc[indd[i], :]
    tmp_train_df = tmp_df[~tmp_df.x1.isnull()]

    # tmp_train_df = tmp_train_df[tmp_train_df.x1 > tmp_train_df.x1.mean() * 0.8]

    tmp_val_df = tmp_df[tmp_df.x1.isnull()]

    ind1 = tmp_train_df['u'].tolist() + tmp_train_df['v'].tolist()
    ind2 = tmp_train_df['v'].tolist() + tmp_train_df['u'].tolist()
    val  = tmp_train_df['x1'].tolist() + tmp_train_df['x1'].tolist()

    mx = max(tmp_df['u'].max(), tmp_df['v'].max())


    friend_matrix = csr_matrix((val, (ind1, ind2)), 
                            shape=(mx + 1, mx + 1))
    
    sums = friend_matrix.T.sum(axis=1).A1
    counts = np.diff(friend_matrix.indptr)
    averages = np.divide(sums, counts, out=np.zeros_like(sums), where=counts!=0) * 0.5

    sums = friend_matrix.sum(axis=1).A1
    counts = np.diff(friend_matrix.T.indptr)
    averages_add = np.divide(sums, counts, out=np.zeros_like(sums), where=counts!=0) * 0.5

    # friend_matrix = csr_matrix(friend_matrix.multiply(friend_matrix > averages))
    # friend_matrix.eliminate_zeros()

    friend_matrix_ones = friend_matrix.copy()
    friend_matrix_ones.data = np.ones(len(friend_matrix_ones.data))
    

    mult_ones1 = friend_matrix_ones * friend_matrix_ones.T
    mult1 = (friend_matrix * friend_matrix_ones.T) 
    i1, i2 = mult1.nonzero()
    a = mult1.data
    b = np.array(mult_ones1[i1, i2])[0] 
    mult1.data = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
    val_ind1 = tmp_val_df['u'].values
    val_ind2 = tmp_val_df['v'].values
    # val_val  = tmp_val_df['target'].values
    # preds1 = (mult1[val_ind1, val_ind2] + mult1[val_ind2, val_ind1] ) / 2
    
    mult_ones2 = friend_matrix_ones.T * friend_matrix_ones
    mult2 = (friend_matrix.T * friend_matrix_ones) 
    i1, i2 = mult2.nonzero()
    a = mult2.data
    b = np.array(mult_ones2[i1, i2])[0] 
    mult2.data = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
    val_ind1 = tmp_val_df['u'].values
    val_ind2 = tmp_val_df['v'].values
    # val_val  = tmp_val_df['target'].values
    # preds2 = (mult2[val_ind1, val_ind2] + mult2[val_ind2, val_ind1] ) / 2

    list_feats1 += mult1[val_ind1, val_ind2].tolist()[0]
    list_feats2 += mult2[val_ind1, val_ind2].tolist()[0]

    list_feats3 += mult1[val_ind2, val_ind1].tolist()[0]
    list_feats4 += mult2[val_ind2, val_ind1].tolist()[0]

    list_feats5 += mult_ones1[val_ind1, val_ind2].tolist()[0]
    list_feats6 += tmp_val_df.index.tolist()

    list_feats7 += averages[val_ind1].tolist()
    list_feats8 += averages[val_ind2].tolist()

    list_feats9 += averages_add[val_ind1].tolist()
    list_feats10 += averages_add[val_ind2].tolist()
    # preds = (preds1 + preds2) / 2
    # list_true += val_val.tolist()
    # list_preds += (np.array(preds)[0] ).tolist()

  0%|          | 0/20596 [00:00<?, ?it/s]

In [ ]:
import pickle
with open('drive/MyDrive/vkfeats2test.pickle', 'wb') as f:
    pickle.dump([list_feats1, list_feats2, list_feats3, list_feats4, list_feats5, list_feats6, list_feats7, list_feats8, list_feats9, list_feats10], f)

In [ ]:
xxx = test.ego_id.unique()
tk0 = tqdm(enumerate(xxx), total = len(xxx))
list_feats1 = []

list_preds = []
list_true = []
for i, num_ in tk0:
    tmp_df = test.loc[indd[i], :]
    tmp_train_df = tmp_df[~tmp_df.x1.isnull()]

    # tmp_train_df = tmp_train_df[tmp_train_df.x1 > tmp_train_df.x1.mean() * 0.8]

    tmp_val_df = tmp_df[tmp_df.x1.isnull()]

    ind1 = tmp_train_df['u'].tolist() 
    ind2 = tmp_train_df['v'].tolist() 
    val  = (tmp_train_df['x1'] + 1).tolist() 

    mx = max(tmp_df['u'].max(), tmp_df['v'].max())


    friend_matrix = csr_matrix((val, (ind1, ind2)), 
                            shape=(mx + 1, mx + 1))
    
    val_ind1 = tmp_val_df['u'].values
    val_ind2 = tmp_val_df['v'].values

    list_feats1 += friend_matrix[val_ind2, val_ind1].tolist()[0]

  0%|          | 0/20596 [00:00<?, ?it/s]

In [ ]:
import pickle
with open('drive/MyDrive/vkfeats3test.pickle', 'wb') as f:
    pickle.dump([list_feats1], f)

In [ ]:
xxx = test.ego_id.unique()
tk0 = tqdm(enumerate(xxx), total = len(xxx))
list_feats1 = []
list_feats2 = []

list_preds = []
list_true = []
for i, num_ in tk0:
    tmp_df = test.loc[indd[i], :]
    tmp_train_df = tmp_df[~tmp_df.x1.isnull()]

    # tmp_train_df = tmp_train_df[tmp_train_df.x1 > tmp_train_df.x1.mean() * 0.8]

    tmp_val_df = tmp_df[tmp_df.x1.isnull()]

    ind1 = tmp_train_df['u'].tolist() 
    ind2 = tmp_train_df['v'].tolist() 
    val  = (tmp_train_df['x1'] + 1).tolist() 

    mx = max(tmp_df['u'].max(), tmp_df['v'].max())


    friend_matrix = csr_matrix((val, (ind1, ind2)), 
                            shape=(mx + 1, mx + 1))
    
    val_ind1 = tmp_val_df['u'].values
    val_ind2 = tmp_val_df['v'].values

    ind_wo0 = zip(ind1, ind2, val)
    ind_wo0 = [(value1, value2, value3) for value1, value2, value3 in ind_wo0 if value1 != 0 and value2 != 0]
    ind_wo0 = list(zip(*ind_wo0))
    try:
        friend_matrix_wo0 = csr_matrix((ind_wo0[2], (ind_wo0[0], ind_wo0[1])), 
                                shape=(mx + 1, mx + 1))
        
        n_comps, labels = connected_components(csgraph=friend_matrix_wo0, directed=True, return_labels=True)
        n_comps -= 1 # remove self
        cnt = Counter(labels)
        labels = np.array([cnt[l] for l in labels])

    except:
        labels = np.ones(mx + 1)

    val_ind1 = tmp_val_df['u'].values
    val_ind2 = tmp_val_df['v'].values


    list_feats1 += labels[val_ind1].tolist()
    list_feats2 += labels[val_ind2].tolist()

  0%|          | 0/20596 [00:00<?, ?it/s]

In [ ]:
import pickle
with open('drive/MyDrive/vkfeats4test.pickle', 'wb') as f:
    pickle.dump([list_feats1, list_feats2], f)

In [ ]:
xxx = test.ego_id.unique()
tk0 = tqdm(enumerate(xxx), total = len(xxx))
list_feats1 = []
list_feats2 = []
list_feats3 = []
list_feats4 = []
list_feats5 = []
list_feats6 = []

list_preds = []
list_true = []
for i, num_ in tk0:
    tmp_df = test.loc[indd[i], :]
    tmp_train_df = tmp_df[~tmp_df.x1.isnull()]

    # tmp_train_df = tmp_train_df[tmp_train_df.x1 > tmp_train_df.x1.mean() * 0.8]

    tmp_val_df = tmp_df[tmp_df.x1.isnull()]

    ind1 = tmp_train_df['u'].tolist() 
    ind2 = tmp_train_df['v'].tolist() 
    val  = (tmp_train_df['x1'] ).tolist() 

    mx = max(tmp_df['u'].max(), tmp_df['v'].max())


    friend_matrix = csr_matrix((val, (ind1, ind2)), 
                            shape=(mx + 1, mx + 1))
    
    friend_matrix_ones = friend_matrix.copy()
    friend_matrix_ones.data = np.ones(len(friend_matrix_ones.data))

    sm = friend_matrix_ones.sum(0).A1

    val_ind1 = tmp_val_df['u'].values
    val_ind2 = tmp_val_df['v'].values

    max_ = friend_matrix.max(0).toarray()[0]

    list_feats1 += max_[val_ind1].tolist()
    list_feats2 += max_[val_ind2].tolist()

    max_ = friend_matrix.T.max(0).toarray()[0]

    list_feats3 += max_[val_ind1].tolist()
    list_feats4 += max_[val_ind2].tolist()

    list_feats5 += sm[val_ind1].tolist()
    list_feats6 += sm[val_ind2].tolist()

  0%|          | 0/20596 [00:00<?, ?it/s]

In [ ]:
import pickle
with open('drive/MyDrive/vkfeats5test.pickle', 'wb') as f:
    pickle.dump([list_feats1, list_feats2, list_feats3, list_feats4, list_feats5, list_feats6], f)

In [ ]:
with open('drive/MyDrive/vkfeats1test.pickle', 'rb') as f:
    list_feats1, list_feats2, list_feats3, list_feats4, list_feats5, list_feats6, list_feats7, list_feats8, list_feats9, list_feats10 = pickle.load(f)

i = 0
for l in [list_feats1, list_feats2, list_feats3, list_feats4, list_feats5, list_feats7, list_feats8, list_feats9, list_feats10]:
    test[f'feat_{i}'] = l
    i += 1

with open('drive/MyDrive/vkfeats2test.pickle', 'rb') as f:
    list_feats1, list_feats2, list_feats3, list_feats4, list_feats5, list_feats6, list_feats7, list_feats8, list_feats9, list_feats10 = pickle.load(f)
    
for l in [list_feats1, list_feats2, list_feats3, list_feats4, list_feats5, list_feats7, list_feats8, list_feats9, list_feats10]:
    test[f'feat_{i}'] = l
    i += 1

with open('drive/MyDrive/vkfeats3test.pickle', 'rb') as f:
    [list_feats1] = pickle.load(f)
    
for l in [list_feats1]:
    test[f'feat_{i}'] = l
    i += 1

In [ ]:
i = 25
with open('drive/MyDrive/vkfeats4test.pickle', 'rb') as f:
    [list_feats1, list_feats2] = pickle.load(f)
    
for l in [list_feats1, list_feats2]:
    test[f'feat_{i}'] = l
    i += 1

test[f'feat_35'] =  test[f'feat_25'] -  test[f'feat_26']

i = 45
with open('drive/MyDrive/vkfeats5test.pickle', 'rb') as f:
    [list_feats1, list_feats2, list_feats3, list_feats4, list_feats5, list_feats6] = pickle.load(f)
    
for l in [list_feats1, list_feats2, list_feats3, list_feats4, list_feats5, list_feats6]:
    test[f'feat_{i}'] = l
    i += 1

In [ ]:
attr = pd.read_csv('attr.csv')

In [ ]:
ss = pd.read_csv('submission.csv')
ss['subm'] = 1

In [ ]:
test = test.merge(ss[['ego_id', 'u', 'v',  'subm']], on = ['ego_id', 'u', 'v'], how = 'left')
test = test[~test.subm.isnull()].reset_index(drop = True)

In [ ]:
test = test.merge(attr[['ego_id', 'u', 'age', 'sex']], on = ['ego_id', 'u'], how = 'left')
test = test.merge(attr[['ego_id', 'u', 'age', 'sex']], left_on = ['ego_id', 'v'], right_on = ['ego_id', 'u'], how = 'left',  suffixes=('_u', '_v'))

In [ ]:
test['f3'] = test['feat_0'] + test['feat_1'] + test['feat_2'] + test['feat_3']

In [ ]:
train_cols = ['t', 'x2', 'x3', 'feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_25', 'feat_26', 'feat_35', 'feat_45', 'feat_46', 'feat_47', 'feat_48', 'feat_49', 'feat_50', 'age_u', 'sex_u', 'age_v', 'sex_v', 'f3']

In [ ]:
import pickle
with open('drive/MyDrive/vkbst.pickle', 'wb') as f:
    bst_list = pickle.load(f)

In [ ]:
bst_list = []
score_list = []
preds = []
# preds_train = np.zeros(len(train))
for i, bst in enumerate(bst_list):
    # pool_train = Pool(train.loc[train_index, train_cols].values, label= train.loc[train_index, 'target'].values)
    # pool_eval = Pool(train.loc[test_index, train_cols].values, label= train.loc[test_index, 'target'].values )
    # bst = catboost.train(pool_train, params_cat, eval_set = pool_eval, iterations = 2000, early_stopping_rounds = 800, verbose = 100)
    # score_list += [bst.get_best_score()['validation']['RMSE']]
    # preds_train[test_index] = bst.predict(train[train_cols][test_index].values) ** 2
    preds += [bst.predict(test[train_cols].values)]
    # bst_list += [bst]

In [ ]:
test['predict'] = np.mean(preds, 0)

In [ ]:
predict_ss = test.groupby(['ego_id', 'u_u', 'v'])['predict'].mean().reset_index()

In [ ]:
ss = ss.merge(predict_ss, left_on = ['ego_id', 'u', 'v'], right_on = ['ego_id', 'u_u', 'v'], how = 'left')
ss['x1'] = ss['predict']

In [ ]:
ss[['ego_id', 'u', 'v', 'x1']].to_csv('predict.csv', index = None)